In [3]:
import pandas as pd
import numpy as np
import matplotlib

### 1. Badanie rezystancji uzwojeń transformatora.

Konwersja danych z miernika analogowego:

In [18]:
conversion = lambda zakres, max_l_dz, wskazanie: (zakres/max_l_dz) * wskazanie
conversion(300,300,230)

230.0

In [14]:
l = [2.050,2,1.025,6.172,2,1.025]
n_pomiar = 1
data = [l] * n_pomiar


df1 = pd.DataFrame(data,index=pd.Index([f'Pomiar {i+1}' for i in range(n_pomiar)], 
                                      name='Wyniki pomiarów:'),
                  columns=pd.MultiIndex.from_product([['Uzwojenie gorne', 'Uzwojenie dolne'], 
                                                      ['U[V]', 'I[A]', 'R[ohm]']]))
df1 = df1.style.set_caption('Tabela 1. Wyniki pomiarów dokonanych podczas badania rezystancji uzwojenia transformatora.')


df1

Wartość średnia rezystancji $R_{1,śr} = 1.025[\Omega]$



### 2. Badanie przekładni napięciowej i prądowej transformatora.

In [34]:
l = [0,0,0] # test samples
n_samples = 4
data = [l] * n_samples

gear = lambda sr1, sr2: sr1/sr2 # calc for U sr1/sr2 , for I sr2/sr1

df2 = pd.DataFrame(data, index=pd.Index([f'U_{i+1}[V]' if i < 2 else f'I_{i-1}[A]' for i in range(n_samples)], 
                                      name='Wyniki pomiarów:'),
                  columns=["Pomiar1", "Uśr[V]", "przekładnia[V][A]"])

df2.iloc[0,0] = 230
df2.iloc[1,0] = 135
df2.iloc[2,0] = 2.5
df2.iloc[3,0] = 4
df2.iloc[0,1] = df2.iloc[0,0]
df2.iloc[1,1] = df2.iloc[1,0]
df2.iloc[2,1] = df2.iloc[2,0]
df2.iloc[3,1] = df2.iloc[3,0] 
df2.iloc[0,2] = gear(df2.iloc[0,0], df2.iloc[1,0])
df2.iloc[1,2] = gear(df2.iloc[0,0], df2.iloc[1,0])
df2.iloc[2,2] = gear(df2.iloc[3,0], df2.iloc[2,0])
df2.iloc[3,2] = gear(df2.iloc[3,0], df2.iloc[2,0])
df2 = df2.style.set_caption('Tabela 2. Wyniki pomiarów przekładni napięciowej i prądowej transformatora.')
df2




,Pomiar1,Uśr[V],przekładnia[V][A]
Wyniki pomiarów:,,,
U_1[V],230.000000,230.000000,1.703704
U_2[V],135.000000,135.000000,1.703704
I_1[A],2.500000,2.500000,1.600000
I_2[A],4.000000,4.000000,1.600000


### 3. Badanie stanu jałowego.

Dla obliczenia poniższych wielkości potrzebujemy skorzystać z poniższych wzorów:<p>
### $cos\phi_{0}=\frac {P_{0}}{U_{0} * I_{0}}$
### $I_{cz}=I_{0}*cos\phi_{0}$
### $I_{\mu}=I_{0}*sin\phi_{0}$  

In [41]:
l = [1,2,3,0,0,0]
data = [[230, 0.168, 7.6],
                [211, 0.121, 6.4],
                [220, 0.14, 6.9],
                [199, 0.1, 5.6],
                [190, 0.089, 5.2],
                [180, 0.077, 4.7],
                [170, 0.068, 4.3],
                [159, 0.06, 3.8],
                [150, 0.054, 3.4],
                [140, 0.041, 3.1],]

n_samples = len(data)

cos_fi_0 = lambda p_0, u_0, i_0: p_0/(u_0 * i_0)

i_m = lambda i_0, sin_fi_0: i_0 * sin_fi_0

i_cz = lambda i_0, cos_fi_0: i_0 * cos_fi_0





df3 = pd.DataFrame(data, columns=["U_0[V]", "I_0[A]", "P_0[W]"],
                  index=np.arange(n_samples)+1)

df3["cos fi_0"] = cos_fi_0(df3["P_0[W]"], df3["U_0[V]"], df3["I_0[A]"])

# df3["Im[A]"] = i_m(df3["I_0[A]"], ) # here we must conver cos fo to sin fi!!

df3["Icz[A]"] = i_cz(df3["I_0[A]"], df3["cos fi_0"])
df3 = df3.style.set_caption("Tabela 3. Wyniki pomiarów charakterystyk stanu jałowego")
df3

# TO DO:

# add calculation for cos fi ect.!!


,U_0[V],I_0[A],P_0[W],cos fi_0,Icz[A]
1,230,0.168000,7.600000,0.196687,0.033043
2,211,0.121000,6.400000,0.250676,0.030332
3,220,0.140000,6.900000,0.224026,0.031364
4,199,0.100000,5.600000,0.281407,0.028141
5,190,0.089000,5.200000,0.307510,0.027368
6,180,0.077000,4.700000,0.339105,0.026111
7,170,0.068000,4.300000,0.371972,0.025294
8,159,0.060000,3.800000,0.398323,0.023899
9,150,0.054000,3.400000,0.419753,0.022667
10,140,0.041000,3.100000,0.540070,0.022143


### 4. Badanie stanu obciążenia

Dla obliczenia poniższych wielkości potrzebujemy skorzystać z poniższych wzorów:<p>
### $P_{2}=U_{2} * I_{2}$
### $cos\phi_{1}=\frac {P_{1}}{U_{1} * I_{1}}$
### $\eta=\frac {P_{2}}{P_{1}} * 100\%$  

In [41]:
l = list(range(9))
measutements = [[],
                [],]
l[-4:] = [0] * 4
n_samples = 12
data = [l] * n_samples

df4 = pd.DataFrame(data, columns=["U_1[V]", "I_1[A]", "P_1[W]", "U_2[V]", "I_2[A]", "P_2[W]", "cos_fi_1",
                                 "n[%]", "I_2/I_1[-]"])

p_2 = lambda u_2, i_2: u_2 * i_2

cos_fi_1 = lambda p_1, u_1, i_1: p_1/(u_1 * i_1)

n = lambda p_2, p_1: (p_2/p_1) * 100

df4=df4.style.set_caption("Tabela 4. Wyniki pomiaru stanu obciążenia transformatora")

df4["P_2[W]"] = p_2(df4["U_2[V]"])
df4


,U_1[V],I_1[A],P_1[W],U_2[V],I_2[A],P_2[W],cos_fi_1,n[%],I_2/I_1[-]
0,0,1,2,3,4,0,0,0,0
1,0,1,2,3,4,0,0,0,0
2,0,1,2,3,4,0,0,0,0
3,0,1,2,3,4,0,0,0,0
4,0,1,2,3,4,0,0,0,0
5,0,1,2,3,4,0,0,0,0
6,0,1,2,3,4,0,0,0,0
7,0,1,2,3,4,0,0,0,0
8,0,1,2,3,4,0,0,0,0
9,0,1,2,3,4,0,0,0,0


### 5. Badanie stanu zwarcia.

Dla obliczenia poniższych wielkości potrzebujemy skorzystać z poniższych wzorów:<p>
### $cos\phi_{z}=\frac {P_{z}}{U_{z} * I_{z}},$
### $R_{z}= \frac {P_{z}}{I_{z}^2},$
### $X_{z}= \sqrt{Z_{z}^2 - R_{z}^2},$
### $Z_{z}=\frac {U_{z}}{I_{z}},$
### $P_{2}=R_{2,śr}*I_{2}^2$
    

In [56]:
l = list(range(1,10)) # test data
l[-6:] = [0] * 6
n_samples = 8
data = [l] * n_samples

cos_fi_z = lambda p_z, u_z, i_z: p_z/(u_z * i_z)

r_z = lambda p_z, i_z: p_z / (i_z**2)

z_z = lambda u_z, i_z: u_z/i_z

x_z = lambda z_z, r_z: (z_z**2 - r_z**2)**.5

p_2 = lambda r_2_sr, i_2: r_2_sr * i_2**2 

columns = ["U_z[V]", "I_z[A]", "P_z[W]", "cos_fi_z", "R_z[ohm]", "X_z[ohm]", "Z_z[ohm]", "I_2[A]", "P_2[W]"]
df5 = pd.DataFrame(data, columns=columns, index=np.arange(n_samples)+1)

df5["cos_fi_z"] = cos_fi_z(df5["P_z[W]"], df5["U_z[V]"], df5["I_z[A]"])

df5["R_z[ohm]"] = r_z(df5["P_z[W]"], df5["I_z[A]"])

df5["Z_z[ohm]"] = z_z(df5["U_z[V]"], df5["I_z[A]"])

df5["X_z[ohm]"] = x_z(df5["Z_z[ohm]"], df5["R_z[ohm]"])

# TO OD
# calc p_2 
# I_2 from we can get ?


df5=df5.style.set_caption("Tabela 5. Tabela wyników pomiarów stanu zwarcia normalnego")
df5


,U_z[V],I_z[A],P_z[W],cos_fi_z,R_z[ohm],X_z[ohm],Z_z[ohm],I_2[A],P_2[W]
1,1,2,3,1.500000,0.750000,nan,0.500000,0,0
2,1,2,3,1.500000,0.750000,nan,0.500000,0,0
3,1,2,3,1.500000,0.750000,nan,0.500000,0,0
4,1,2,3,1.500000,0.750000,nan,0.500000,0,0
5,1,2,3,1.500000,0.750000,nan,0.500000,0,0
6,1,2,3,1.500000,0.750000,nan,0.500000,0,0
7,1,2,3,1.500000,0.750000,nan,0.500000,0,0
8,1,2,3,1.500000,0.750000,nan,0.500000,0,0


In [58]:
x_z(3, 1)

2.8284271247461903

# TO DO
    6. Opracowanie wyników.
    a) Na podstawie wyników pomiarów obliczyć pozostałe wartości
    b) Wykresy wykonać na PAPIERZE MILIMETROWYM!
    c) Wyznaczyć charakterystyki stanu jałowego, tj. 
    d) Wyznaczyć charakterystyki stanu obciążenia, tj. 
    e) Wyznaczyć charakterystyki zwarciowe, tj. , 
    f) Przeprowadzić dyskusję wyników oraz podać wnioski wg wytycznych

In [37]:
l = []

[0, 1, 2, 3, 0, 0, 0, 0]